### Implement MotifEnumeration

https://rosalind.info/problems/ba2a/

In [13]:
from itertools import product

def HammingDistance(p, q):
        '''
        Computes the Hamming distance between two strings.
        Args:
            p: A string.
            q: A string of the same length as p.
        Returns:
            The Hamming distance between p and q.
        '''
        return sum(pc != qc for pc, qc in zip(p, q))

def Neighbors(pattern, d):
        if d == 0:
            return {pattern}
        if len(pattern) == 0:
            return {""}
        neighborhood = set()
        suffix_neighbors = Neighbors(pattern[1:], d)
        for text in suffix_neighbors:
            if HammingDistance(pattern[1:], text) < d:
                for nucleotide in "ACGT":
                    neighborhood.add(nucleotide + text)
            else:
                neighborhood.add(pattern[0] + text)
        return neighborhood

def MotifEnumeration(Dna, k, d):
    '''Finds all (k, d)-motifs in a collection of DNA strings.
    A (k, d)-motif is a k-mer that appears in each string from Dna with at most d mismatches.
    Args:
        Dna: A list of DNA strings.
        k: The length of the k-mers to find.
        d: The maximum number of mismatches allowed.
    Returns:
        A set of all (k, d)-motifs in Dna.
    '''
    patterns = set()
    for seq in Dna:
        for i in range(len(seq) - k + 1):
            pattern = seq[i:i + k]
            neighborhood = Neighbors(pattern, d)
            patterns.update(neighborhood)

    result = set()
    for pattern in patterns:
        if all(any(HammingDistance(pattern, seq[i:i + k]) <= d for i in range(len(seq) - k + 1)) for seq in Dna):
            result.add(pattern)

    return result

In [14]:
file_path = 'data/rosalind_ba2a.txt'
with open(file_path, 'r') as f:
    lines = f.read().strip().split('\n')
    k, d = map(int, lines[0].split())
    Dna = lines[1:]
result = MotifEnumeration(Dna, k, d)
print(' '.join(result))

ATCCG ACCCG AACCG AGCCG


### Find a Median String

https://rosalind.info/problems/ba2b/

In [15]:
from itertools import product

def HammingDistance(p, q):
    return sum(pc != qc for pc, qc in zip(p, q))

def DistanceBetweenPatternAndStrings(pattern, Dna):
    k = len(pattern)
    total_distance = 0
    for seq in Dna:
        min_distance = min(HammingDistance(pattern, seq[i:i + k]) for i in range(len(seq) - k + 1))
        total_distance += min_distance
    return total_distance


def MedianString(Dna, k):
    '''Finds a k-mer that minimizes the distance to a collection of DNA strings.
    The distance between a k-mer and a DNA string is the minimum Hamming distance between the
    k-mer and any k-mer in the DNA string. The distance between a k-mer and a collection of DNA strings
    is the sum of the distances between the k-mer and each string in the collection.
    Args:
        Dna: A list of DNA strings.
        k: The length of the k-mers to find.
    Returns:
        A k-mer that minimizes the distance to Dna.
    '''
    min_distance = float('inf')
    median = None
    for pattern in product("ACGT", repeat=k):
        pattern_str = ''.join(pattern)
        distance = DistanceBetweenPatternAndStrings(pattern_str, Dna)
        if distance < min_distance:
            min_distance = distance
            median = pattern_str

    return median

In [16]:
with open('data/rosalind_ba2b.txt', 'r') as f:
    lines = f.read().strip().split('\n')
    k = int(lines[0])
    Dna = lines[1:]
median = MedianString(Dna, k)
print(median)

CTCTAG


### Find a Profile-most Probable k-mer in a String

https://rosalind.info/problems/ba2c/

In [17]:
def ProfileMostProbableKmer(text, k, profile):
    '''Finds the most probable k-mer in a string given a profile matrix.
    Args:
        text: A DNA string.
        k: The length of the k-mer to find.
        profile: A dictionary representing the profile matrix with keys 'A', 'C', 'G', 'T'.
                 Each key maps to a list of probabilities for each position in the k-mer.
    Returns:
        The most probable k-mer in text according to the profile matrix.
    '''
    max_prob = -1
    most_prob_kmer = text[0:k]  # Default to the first k-mer
    for i in range(len(text) - k + 1):
        kmer = text[i:i + k]
        prob = 1
        for j, nucleotide in enumerate(kmer):
            prob *= profile[nucleotide][j]
        if prob > max_prob:
            max_prob = prob
            most_prob_kmer = kmer
    return most_prob_kmer

In [18]:
with open('data/rosalind_ba2c.txt', 'r') as f:
    lines = f.read().strip().split('\n')
    text = lines[0]
    k = int(lines[1])
    profile_lines = lines[2:6]
    profile = {}
    nucleotides = ['A', 'C', 'G', 'T']
    for nucleotide, line in zip(nucleotides, profile_lines):
        profile[nucleotide] = list(map(float, line.split()))
most_prob_kmer = ProfileMostProbableKmer(text, k, profile)
print(most_prob_kmer)

GACTTTG


### Implement GreedyMotifSearch

https://rosalind.info/problems/ba2d/

In [26]:
from collections import Counter

def score(motifs):
    """Compute the score of a set of motifs (lower is better)."""
    k = len(motifs[0])
    t = len(motifs)
    score_val = 0
    for j in range(k):
        col = [motifs[i][j] for i in range(t)]
        counts = Counter(col)
        max_count = max(counts.values())
        score_val += t - max_count
    return score_val


def profile(motifs):
    """Generate a profile matrix (no pseudocounts)."""
    k = len(motifs[0])
    t = len(motifs)
    profile = {base: [0] * k for base in "ACGT"}

    for j in range(k):
        for i in range(t):
            base = motifs[i][j]
            profile[base][j] += 1
        total = sum(profile[base][j] for base in "ACGT")
        for base in "ACGT":
            profile[base][j] /= total
    return profile


def probability(kmer, profile):
    """Compute the probability of a k-mer given a profile matrix."""
    prob = 1.0
    for i, base in enumerate(kmer):
        prob *= profile[base][i]
    return prob


def profile_most_probable_kmer(text, k, profile):
    """Return the most probable k-mer in a string given a profile matrix."""
    max_prob = -1
    best_kmer = text[0:k]
    for i in range(len(text) - k + 1):
        kmer = text[i:i+k]
        prob = probability(kmer, profile)
        if prob > max_prob:
            max_prob = prob
            best_kmer = kmer
    return best_kmer


def greedy_motif_search(Dna, k, t):
    """The GREEDYMOTIFSEARCH algorithm (no pseudocounts)."""
    BestMotifs = [dna[0:k] for dna in Dna]  # initial motifs: first k bases of each string
    first_string = Dna[0]

    for i in range(len(first_string) - k + 1):
        Motif1 = first_string[i:i+k]
        Motifs = [Motif1]
        for j in range(1, t):
            prof = profile(Motifs)
            next_motif = profile_most_probable_kmer(Dna[j], k, prof)
            Motifs.append(next_motif)
        if score(Motifs) < score(BestMotifs):
            BestMotifs = Motifs
    return BestMotifs

In [27]:
with open("data/rosalind_ba2d.txt", "r") as f:
        lines = f.read().strip().split("\n")
        k, t = map(int, lines[0].split())
        Dna = lines[1:]

BestMotifs = greedy_motif_search(Dna, k, t)
print("\n".join(BestMotifs))

GGAGCTACAGTG
GTCTATTGCATA
CAGTCGGCGCAC
GACGAGTCGGTA
GTATAGGCGATC
CCGCCGCAGTCC
CGATCTTACTCC
CGCTCTAGAATA
CACTCTAGAGTG
CCCGCTAGACTC
CACGCGTACCTC
CACGCTTCCGCG
CGCTCGCCAGCC
GTGGATACACTG
CCCGCTAGACTC
CACTCTAGACTG
CCCGCTAGATTA
CCCTCTAACGAA
GGGGCTAGCTCA
CACTCTAGAATC
CACGCTAGAGTG
CACGCTTGGGCG
CGCCCTAGATTA
CGCTCTAGAATG
CTCTCTAGAATC


### Implement GreedyMotifSearch with Pseudocounts

https://rosalind.info/problems/ba2e/

In [30]:
def ProfileWithPseudocounts(motifs):
    profile = {nucleotide: [1] * k for nucleotide in "ACGT"}
    for motif in motifs:
        for j, nucleotide in enumerate(motif):
            profile[nucleotide][j] += 1
    total = len(motifs) + 4
    for nucleotide in profile:
        profile[nucleotide] = [count / total for count in profile[nucleotide]]
    return profile

def Score(motifs):
    score = 0
    for j in range(k):
        counts = {nucleotide: 0 for nucleotide in "ACGT"}
        for motif in motifs:
            counts[motif[j]] += 1
        max_count = max(counts.values())
        score += len(motifs) - max_count
    return score

def GreedyMotifSearch(Dna, k, t):
    '''
    The GREEDYMOTIFSEARCH algorithm with pseudocounts.
    Args:
        Dna: A list of DNA strings.
        k: The length of the k-mers to find.
        t: The number of DNA strings in Dna.
    Returns:
        A list of the best motifs found.
    '''
    best_motifs = [seq[:k] for seq in Dna]
    best_score = Score(best_motifs)

    first_seq = Dna[0]
    for i in range(len(first_seq) - k + 1):
        motifs = [first_seq[i:i + k]]
        for j in range(1, t):
            current_profile = ProfileWithPseudocounts(motifs)
            next_motif = ProfileMostProbableKmer(Dna[j], k, current_profile)
            motifs.append(next_motif)
        current_score = Score(motifs)
        if current_score < best_score:
            best_score = current_score
            best_motifs = motifs

    return best_motifs

In [32]:
with open('data/rosalind_ba2e.txt', 'r') as f:
    lines = f.read().strip().split('\n')
    k, t = map(int, lines[0].split())
    Dna = lines[1:]
BestMotifs = GreedyMotifSearch(Dna, k, t)
print('\n'.join(BestMotifs))

TCCCATCGGTCG
ACCCATCCGTCT
GCCCATCAGTAC
TCCCATCGGTTT
GCCCATCTGTGC
CCCCATCCGTCC
CCCCATCTGTGT
ACCCATCGGTTC
GCCCATCCGTAA
GCCCATCGGTTC
CCCCATCGGTCT
GCCCATCCGTAT
TCCCATCTGTCT
TCCCATCGGTGT
CCCCATCCGTGA
CCCCATCAGTAA
ACCCATCAGTGA
ACCCATCGGTCC
ACCCATCAGTAA
CCCCATCCGTAC
CCCCATCGGTTA
ACCCATCAGTGG
TCCCATCGGTGT
GCCCATCAGTGG
TCCCATCTGTCA


### Implement RandomizedMotifSearch

https://rosalind.info/problems/ba2f/

In [51]:
import random

def score(motifs):
    consensus = ""
    k = len(motifs[0])
    for j in range(k):
        counts = {"A":0, "C":0, "G":0, "T":0}
        for motif in motifs:
            counts[motif[j]] += 1
        consensus += max(counts, key=counts.get)
    score = 0
    for motif in motifs:
        for j in range(k):
            if motif[j] != consensus[j]:
                score += 1
    return score

def profile_with_pseudocounts(motifs):
    k = len(motifs[0])
    t = len(motifs)
    profile = {ch:[1]*(k) for ch in "ACGT"}  # pseudocount=1
    for motif in motifs:
        for j, ch in enumerate(motif):
            profile[ch][j] += 1
    # normalize
    for j in range(k):
        total = sum(profile[ch][j] for ch in "ACGT")
        for ch in "ACGT":
            profile[ch][j] /= total
    return profile

def pr(kmer, profile):
    p = 1.0
    for j, ch in enumerate(kmer):
        p *= profile[ch][j]
    return p

def profile_most_probable_kmer(text, k, profile):
    max_prob = -1
    most_prob_kmer = text[0:k]
    for i in range(len(text) - k + 1):
        kmer = text[i:i+k]
        prob = pr(kmer, profile)
        if prob > max_prob:
            max_prob = prob
            most_prob_kmer = kmer
    return most_prob_kmer

def randomized_motif_search(Dna, k, t):
    # randomly select k-mers
    motifs = []
    for seq in Dna:
        start = random.randint(0, len(seq)-k)
        motifs.append(seq[start:start+k])
    best_motifs = motifs[:]
    while True:
        profile = profile_with_pseudocounts(motifs)
        motifs = [profile_most_probable_kmer(seq, k, profile) for seq in Dna]
        if score(motifs) < score(best_motifs):
            best_motifs = motifs[:]
        else:
            return best_motifs

def repeated_randomized_motif_search(Dna, k, t, N=1000):
    best_motifs = randomized_motif_search(Dna, k, t)
    for _ in range(N-1):
        motifs = randomized_motif_search(Dna, k, t)
        if score(motifs) < score(best_motifs):
            best_motifs = motifs
    return best_motifs


In [53]:
with open("data/rosalind_ba2f.txt", "r") as f:
    lines = f.read().strip().split("\n")
    k, t = map(int, lines[0].split())
    Dna = lines[1:]

best = repeated_randomized_motif_search(Dna, k, t, N=1000)
print("\n".join(best))

CTTGCCGATGATACT
CTTAAGAGACACACT
GCTGGTAGACACACC
GTTGGTAGACACAAC
CTTGCAGGACACACT
CGGAGTAGACACACT
CTCCCTAGACACACT
CTTGGTAGACCACCT
CTTGGTAGACACTGA
CTTGGTAACGACACT
CTTGGTAGACAGCAT
CTTGGTAGAACGACT
CTTCCGAGACACACT
CTTGCGGGACACACT
CTTGGTTCTCACACT
CTTGGTTAGCACACT
CTTGGCTCACACACT
AACGGTAGACACACT
CTTGGTAGTATCACT
CTTGGATAACACACT


### Implement GibbsSampler

https://rosalind.info/problems/ba2g/

In [55]:
import random
from collections import Counter

def score(motifs):
    """Compute the score of a set of motifs (lower is better)."""
    k = len(motifs[0])
    t = len(motifs)
    score_val = 0
    for j in range(k):
        col = [motif[j] for motif in motifs]
        counts = Counter(col)
        max_count = max(counts.values())
        score_val += t - max_count
    return score_val

def profile_with_pseudocounts(motifs):
    """Construct profile with pseudocounts."""
    k = len(motifs[0])
    profile = {ch:[1]*k for ch in "ACGT"}  # pseudocount = 1
    for motif in motifs:
        for j, ch in enumerate(motif):
            profile[ch][j] += 1
    # normalize
    for j in range(k):
        total = sum(profile[ch][j] for ch in "ACGT")
        for ch in "ACGT":
            profile[ch][j] /= total
    return profile

def pr(kmer, profile):
    """Probability of k-mer given profile."""
    p = 1.0
    for j, ch in enumerate(kmer):
        p *= profile[ch][j]
    return p

def profile_randomly_generated_kmer(text, k, profile):
    """Randomly select a k-mer from text weighted by profile probabilities."""
    probabilities = []
    n = len(text) - k + 1
    for i in range(n):
        kmer = text[i:i+k]
        probabilities.append(pr(kmer, profile))
    total = sum(probabilities)
    probabilities = [p/total for p in probabilities]  # normalize
    chosen_index = random.choices(range(n), weights=probabilities, k=1)[0]
    return text[chosen_index:chosen_index+k]

def gibbs_sampler(Dna, k, t, N):
    """Gibbs Sampler for motifs."""
    motifs = []
    for seq in Dna:
        start = random.randint(0, len(seq)-k)
        motifs.append(seq[start:start+k])
    best_motifs = motifs[:]

    for j in range(N):
        i = random.randint(0, t-1)  # pick one sequence to update
        motifs_except_i = motifs[:i] + motifs[i+1:]
        profile = profile_with_pseudocounts(motifs_except_i)
        motifs[i] = profile_randomly_generated_kmer(Dna[i], k, profile)
        if score(motifs) < score(best_motifs):
            best_motifs = motifs[:]

    return best_motifs

def repeated_gibbs_sampler(Dna, k, t, N, runs=20):
    """Run GibbsSampler multiple times and return the best overall motifs."""
    best_motifs_overall = gibbs_sampler(Dna, k, t, N)
    best_score_overall = score(best_motifs_overall)
    for _ in range(runs-1):
        motifs = gibbs_sampler(Dna, k, t, N)
        s = score(motifs)
        if s < best_score_overall:
            best_score_overall = s
            best_motifs_overall = motifs[:]
    return best_motifs_overall


In [56]:
with open("data/rosalind_ba2g.txt", "r") as f:
    lines = f.read().strip().split("\n")
    k, t, N = map(int, lines[0].split())
    Dna = lines[1:]
best = repeated_gibbs_sampler(Dna, k, t, N, runs=20)
print("\n".join(best))

AATATCAAACAGGCC
GGTTCCAAACTGTAC
GGTATTAAAGATTAC
GGTATATGACTGTAC
GGTATTAAACTGGTT
GGTATTATCATGTAC
GGTATTAACTGGTAC
GGTTCCAAACTGTAC
GGTATTTGGCTGTAC
GGTAGGTAACTGTAC
GGTATACCACTGTAC
TTCATTAAACTGTAC
GGTATTCGCCTGTAC
GGGGCTAAACTGTAC
GAGGTTAAACTGTAC
GGTATTAAACTCGCC
GGTATTAAACATGAC
GGTACCGAACTGTAC
TTTATTAAACTGTAA
CGTATTAAACTGTGT
